In [8]:
import warnings
warnings.filterwarnings('ignore')
import flowermd
import hoomd
import gsd
import matplotlib.pyplot as plt
import numpy as np
import gsd.hoomd
from flowermd.base import Pack, Simulation
from flowermd.library import OPLS_AA_PPS,PPS
from flowermd.utils import get_target_box_mass_density
import unyt as u
import hoomd

gpu = hoomd.device.GPU()

In [6]:
pps_chains = PPS(lengths=[15,17,22], num_mols=[3,2,2])
system = Pack(molecules=[pps_chains], density=0.1)
system.apply_forcefield(
    r_cut=2.5,
    force_field=OPLS_AA_PPS(),
    auto_scale=True,
    remove_hydrogens=True,
    remove_charges=True)

No charged group detected, skipping electrostatics.


In [7]:
gsd_path=('ua-pps-trajectory0.gsd')
log_path=('ua-pps-log0.txt')
sim = Simulation.from_system(
    system=system,
    gsd_write_freq=int(1e5),
    gsd_file_name=gsd_path,
    log_file_name=log_path,
    dt=0.001,
    log_write_freq=int(1e5),
    device=gpu,
)
#target_box = get_target_box_mass_density(density=system.density, mass=system.mass.to("g"))
#lj_sim.run_update_volume(final_box_lengths=target_box,kT=5.0,n_steps=1e4,tau_kt=5*lj_sim.dt,period=10)
sim.run_NVT(n_steps=int(1e7), kT=5.0, tau_kt=10*sim.dt)
sim.flush_writers()
sim.save_restart_gsd("pps_restart.gsd")

Initializing simulation state from a gsd.hoomd.Frame.
Step 100000 of 10000000; TPS: 9011.76; ETA: 18.3 minutes
Step 200000 of 10000000; TPS: 9479.12; ETA: 17.2 minutes
Step 300000 of 10000000; TPS: 9638.62; ETA: 16.8 minutes
Step 400000 of 10000000; TPS: 9721.32; ETA: 16.5 minutes
Step 500000 of 10000000; TPS: 9780.4; ETA: 16.2 minutes
Step 600000 of 10000000; TPS: 9816.09; ETA: 16.0 minutes
Step 700000 of 10000000; TPS: 9837.47; ETA: 15.8 minutes
Step 800000 of 10000000; TPS: 9856.9; ETA: 15.6 minutes
Step 900000 of 10000000; TPS: 9872.56; ETA: 15.4 minutes
Step 1000000 of 10000000; TPS: 9880.8; ETA: 15.2 minutes
Step 1100000 of 10000000; TPS: 9890.52; ETA: 15.0 minutes
Step 1200000 of 10000000; TPS: 9895.3; ETA: 14.8 minutes
Step 1300000 of 10000000; TPS: 9898.77; ETA: 14.6 minutes
Step 1400000 of 10000000; TPS: 9901.11; ETA: 14.5 minutes
Step 1500000 of 10000000; TPS: 9908.31; ETA: 14.3 minutes
Step 1600000 of 10000000; TPS: 9912.23; ETA: 14.1 minutes
Step 1700000 of 10000000; TPS: 

In [10]:
from cmeutils.sampling import is_equilibrated
shrink_cut = 10
log = np.genfromtxt(log_path, names=True)
pe = log["mdcomputeThermodynamicQuantitiespotential_energy"]
# equil: bool (True or False), t0: int (starting data point that you can sample from), neff: number of data points in equilibrated region
print(pe[shrink_cut:])
equil, t0, g, neff = is_equilibrated(data=pe[shrink_cut:], threshold_neff=50) 
if equil:
    print("True")
else:
    # If it's not equilibrated, job.doc.equilibrated stays false, and the "run-longer" operation in project.py will run next time we submit this job
    print("False")

[9203.39 8932.86 9084.6  9360.99 9330.16 9248.34 8945.21 9520.46 9229.16
 8944.75 9010.49 9517.01 9038.07 8870.66 9140.81 9543.05 9048.4  8897.06
 8591.42 8926.73 9214.55 9215.8  9168.46 9208.59 9124.16 9198.58 9092.48
 9021.91 9349.28 9112.78 9104.4  9020.81 8976.22 9157.4  8818.23 9329.62
 9105.75 9574.16 9053.49 9002.99 8970.11 9035.53 9172.47 9190.03 9126.92
 8728.04 9308.22 9190.9  9110.82 8793.14 8975.25 9288.21 8835.22 8977.78
 9023.84 9326.07 8840.74 8772.68 9028.49 8953.18 9345.78 9044.91 8806.64
 9169.45 9275.77 9242.05 9084.35 9354.08 9027.57 9286.8  9152.93 8524.66
 8815.65 8945.55 8714.06 8875.   9277.   8798.55 9169.83 9298.87 8923.45
 9188.67 8659.64 9004.06 9323.93 8937.69 8961.04 8971.34 9146.55 9318.99
 9161.68]
True


In [4]:
import numpy as np
import mbuild as mb

from grits import CG_System
from grits.utils import amber_dict

PPS_smiles = "c1ccc(S)cc1"
gsdfile = "ua-pps-trajectory0.gsd"
system = CG_System(
    gsdfile,
    add_hydrogens=True,
    beads={"A": PPS_smiles},
    conversion_dict=amber_dict,
)

Added 62 hydrogens.
Added 90 hydrogens.
Added 70 hydrogens.


In [5]:
cg_gsd = "PPS-cg.gsd"
system.save(cg_gsd)

IndexError: index 34 is out of bounds for axis 0 with size 34

In [6]:
system.save_mapping(filename='test.json')

Mapping saved to test.json


In [9]:
from grits import CG_Compound
from grits.utils import amber_dict

pps_chains = PPS(lengths=[15,17,22], num_mols=[3,2,2])

In [10]:
pps_chains.coarse_grain(beads={"A": PPS_smiles})

In [15]:
pps_chains.molecules

[<CG_Compound: 15 beads pos=(-0.0000, 0.1355,-4.4410), 14 bonds>,
 <CG_Compound: 15 beads pos=(-0.0000, 0.1355,-4.4410), 14 bonds>,
 <CG_Compound: 15 beads pos=(-0.0000, 0.1355,-4.4410), 14 bonds>,
 <CG_Compound: 17 beads pos=(-0.0000, 0.1533,-5.0755), 16 bonds>,
 <CG_Compound: 17 beads pos=(-0.0000, 0.1533,-5.0755), 16 bonds>,
 <CG_Compound: 22 beads pos=(-0.0000, 0.1978,-6.6619), 21 bonds>,
 <CG_Compound: 22 beads pos=(-0.0000, 0.1978,-6.6619), 21 bonds>]

In [12]:
pps_chains

AttributeError: 'PPS' object has no attribute 'save'